In [ ]:
# !pip install git+https://github.com/google/BIG-bench.git # This may take a few minutes
# !pip install datasets==2.21.0
# !pip install transformer_lens
!pip install -r /content/drive/MyDrive/University/CoT/geometry-of-truth/requirements.txt
!pip install nnsight

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip list

Package                          Version
-------------------------------- -------------------
absl-py                          1.4.0
accelerate                       0.34.2
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.6
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.16
albumentations                   1.4.15
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.19.0
astropy                          6.1.3
astropy-iers-data                0.2024.9.23.0.31.43
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread                        3.0.1
autograd                 

# Imports

In [ ]:
import torch
import json
import configparser
from nnsight import LanguageModel
import huggingface_hub
import pandas as pd
from transformers import AutoTokenizer

# Constants

In [ ]:
DATA_PATH = "./data/com2sense.json"
CONFIG_PATH = "./geometry-of-truth/config.ini"
MODEL_NAME = "llama-3.2-1b"
N_SHOTS = 5
BATCH_SIZE = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
remote = False

print(f"Executing on device {device}")

Executing on device cpu


# Load data

In [ ]:
def load_data(n_shots: int, data_path: str, data_key: str = "examples", lookup_key: str = "pair_id_lookup") -> tuple:
    with open(data_path, "r") as file:
        dataset = json.load(file)

    examples = dataset[data_key]
    pair_id_lookup = dataset[lookup_key]
    dataset = pd.DataFrame(examples)
    train = dataset.sample(n_shots)
    test = dataset.drop(train.index)

    return train, test, pair_id_lookup

train, test, pair_id_lookup = load_data(N_SHOTS, DATA_PATH)

# Load model

In [ ]:
huggingface_hub.login()

In [ ]:
config = configparser.ConfigParser()
config.read(CONFIG_PATH)

def load_model(model_name: str, device='remote') -> LanguageModel:
    print(f"Loading model {model_name}...")
    weights_directory = config[model_name]['weights_directory']
    model = LanguageModel(weights_directory, torch_dtype=torch.bfloat16, device_map=device)
    model.tokenizer = AutoTokenizer.from_pretrained(weights_directory)
    return model

model = load_model(MODEL_NAME, device=device)
model.tokenizer.padding_side = "right"
model.tokenizer.pad_token = model.tokenizer.eos_token
print(model)

Loading model llama-3.2-1b...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

# Train model with prompt

In [ ]:
def train_model(model: LanguageModel, prompts: pd.DataFrame, prompt_key: str = "sent", label_key: str = "label") -> tuple:
    """
    Trains the model on the given prompts. Will return the past key value pairs representing this context.
    args:
        model: The model to train.
        prompts: The prompts to train on.
        prompt_key: The key in the prompts dataframe that contains the prompt.
        label_key: The key in the prompts dataframe that contains the label.
    returns: tuple of shape (n_layers, 2) that contains the past key/value (torch.Tensor) with shape (batch_size, num_attention_heads, seq_len, head_dim)
    """
    prompt = ""
    for index, example in prompts.iterrows():
        prompt += example[prompt_key]
        label = example[label_key]

        if bool(label):
            prompt += " TRUE\n"
        else:
            prompt += " FALSE\n"
    with torch.no_grad():
        with model.trace(prompt, output_hidden_states=True, use_cache=True) as tracer:
            output = tracer.output.save()
    return output["past_key_values"], prompt

past_key_values, train_prompt = train_model(model, train)

# Obtain activation difference from testing set

In [ ]:
def obtain_activation_diff(model: LanguageModel, queries: pd.DataFrame, batch_size: int, past_key_values: tuple, train_prompt: str, prompt_key: str = "sent") -> torch.Tensor:
    """
    Obtains the activation difference between the model's predictions of true or false for the given queries.
    args:
        model: The model to use.
        queries: The queries to use.
        batch_size: The batch size to use.
        past_key_values: The past key values to use.
        train_prompt: The prompt used to train the model.
        prompt_key: The key in the queries dataframe that contains the prompt.
    returns: The activation difference between the model's predictions for the given queries.
    """
    true_idx, false_idx = model.tokenizer.encode(' TRUE')[-1], model.tokenizer.encode(' FALSE')[-1]
    diffs = []
    for batch_idx in range(0, len(queries), batch_size):
        batch = queries.iloc[batch_idx : batch_idx + batch_size][prompt_key].tolist()
        batch = [train_prompt + query for query in batch]

        # prepare past_key_values
        pkv_batch = tuple(
            (
              past_key_values[layer][0].expand(batch_size, *past_key_values[layer][0].shape[1:]),
              past_key_values[layer][1].expand(batch_size, *past_key_values[layer][0].shape[1:])
            ) for layer in range(len(past_key_values))
        )
        batch_tokens = model.tokenizer(batch, return_tensors="pt", padding=True, truncation=True)["input_ids"]

        print(pkv_batch[0][0].shape)
        print(batch_tokens.shape)
        batch_lens = [len(model.tokenizer.encode(query, add_special_tokens=False)) for query in batch]
        with torch.no_grad():
            with model.trace(batch_tokens) as tracer:
                logits = model.lm_head.output.save()
                logits = logits[torch.arange(len(batch)), torch.tensor(batch_lens) - 1, :]
                probs = logits.softmax(-1)
                diffs.append((probs[:, true_idx] - probs[:, false_idx]).save())
    diffs = torch.cat([diff.value for diff in diffs])
    return diffs

diffs = obtain_activation_diff(model, test, BATCH_SIZE, past_key_values, train_prompt)

torch.Size([32, 8, 138, 64])
torch.Size([32, 173])


KeyboardInterrupt: 